# 05 - AI-Assisted Experiment Generation

This notebook demonstrates LeeQ's unique AI integration capabilities for automated experiment generation and optimization.

## Learning Objectives
- Understand AI-assisted experiment design
- Learn about automated parameter optimization
- Practice with intelligent calibration workflows
- Explore LLM integration for experiment planning

## Prerequisites
- Complete [04_calibration.ipynb](04_calibration.ipynb)
- Understanding of machine learning concepts (helpful but not required)

## Setup and Configuration

In [ ]:
# Import required modules for AI integration
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Core LeeQ imports
from leeq.chronicle import Chronicle, log_and_record
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.experiments.experiments import ExperimentManager
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon

# Import experiment modules
from leeq.experiments.builtin.basic.calibrations import (
    RabiAmplitudeCalibration,
    MeasurementStatistics
)
from leeq.experiments.builtin.basic.characterizations import (
    T1Measurement,
    T2RamseyMeasurement
)

# Start Chronicle logging
Chronicle().start_log()
log_and_record("ai_integration_tutorial_start", {
    "notebook": "05_ai_integration",
    "timestamp": datetime.now().isoformat()
})

print("AI Integration modules imported successfully!")
print("Chronicle logging started for AI tutorial")

# Create AI-optimized virtual quantum system
manager = ExperimentManager()
manager.clear_setups()

# Create virtual transmon with AI-optimized parameters
ai_qubit = VirtualTransmon(
    name="AIOptimizedQubit",
    qubit_frequency=5042.3,
    anharmonicity=-201,
    t1=85,  # Optimized coherence times
    t2=45,
    readout_frequency=9647.8,
    quiescent_state_distribution=np.asarray([0.88, 0.10, 0.015, 0.005])
)

# Setup high-level simulation
setup = HighLevelSimulationSetup(
    name='AIAssistedSetup',
    virtual_qubits={1: ai_qubit}
)
manager.register_setup(setup)

print(f"AI-optimized quantum system initialized:")
print(f"  Qubit: {ai_qubit.name}")
print(f"  Frequency: {ai_qubit.qubit_frequency} MHz")
print(f"  T1: {ai_qubit.t1} μs, T2: {ai_qubit.t2} μs")

## AI-Assisted Experiment Design

LeeQ's AI integration allows for intelligent experiment design and optimization. AI can:

1. **Generate Experiment Sequences**: Automatically create optimal measurement sequences
2. **Parameter Optimization**: Use machine learning to find optimal control parameters
3. **Adaptive Protocols**: Adjust experiments based on real-time results
4. **Error Mitigation**: Intelligently design error correction sequences
5. **Resource Optimization**: Minimize experimental time while maximizing information gain

### AI-Powered Experiment Generator

The AI system analyzes your quantum system and experimental goals to suggest optimal protocols.

In [ ]:
class AIExperimentGenerator:
    """
    AI-assisted experiment generation and optimization.
    """
    
    def __init__(self, quantum_system):
        self.system = quantum_system
        self.experiment_library = self._initialize_library()
        self.optimization_history = []
        
    def _initialize_library(self):
        """Initialize library of experiment templates."""
        return {
            'characterization': {
                'rabi': {'priority': 'high', 'time_cost': 5, 'info_gain': 0.9},
                't1': {'priority': 'medium', 'time_cost': 10, 'info_gain': 0.7},
                't2_ramsey': {'priority': 'medium', 'time_cost': 8, 'info_gain': 0.6},
                't2_echo': {'priority': 'low', 'time_cost': 12, 'info_gain': 0.5}
            },
            'calibration': {
                'frequency': {'priority': 'high', 'time_cost': 3, 'info_gain': 0.95},
                'amplitude': {'priority': 'high', 'time_cost': 4, 'info_gain': 0.9},
                'drag': {'priority': 'medium', 'time_cost': 15, 'info_gain': 0.6}
            }
        }
        
    def generate_optimal_sequence(self, time_budget=30, objectives=['calibration']):
        """
        Generate optimal experiment sequence given time budget and objectives.
        """
        print(f"\n🤖 AI Experiment Generator")
        print(f"Time budget: {time_budget} minutes")
        print(f"Objectives: {objectives}")
        print("=" * 50)
        
        # Collect candidate experiments
        candidates = []
        for category in objectives:
            if category in self.experiment_library:
                for exp_name, props in self.experiment_library[category].items():
                    candidates.append({
                        'name': exp_name,
                        'category': category,
                        'time': props['time_cost'],
                        'gain': props['info_gain'],
                        'priority': props['priority'],
                        'efficiency': props['info_gain'] / props['time_cost']
                    })
        
        # AI optimization: maximize information gain within time budget
        selected_experiments = self._optimize_sequence(candidates, time_budget)
        
        return selected_experiments
    
    def _optimize_sequence(self, candidates, time_budget):
        """Optimize experiment sequence using greedy algorithm."""
        # Sort by efficiency (info gain per unit time)
        candidates.sort(key=lambda x: x['efficiency'], reverse=True)
        
        selected = []
        remaining_time = time_budget
        total_gain = 0
        
        for exp in candidates:
            if exp['time'] <= remaining_time:
                selected.append(exp)
                remaining_time -= exp['time']
                total_gain += exp['gain']
                
        print(f"\n🎯 Optimized Sequence:")
        for i, exp in enumerate(selected, 1):
            efficiency_score = exp['efficiency'] * 100
            print(f"  {i}. {exp['name'].upper()} ({exp['category']})")
            print(f"     Time: {exp['time']}min, Gain: {exp['gain']:.2f}, Efficiency: {efficiency_score:.0f}%")
            
        print(f"\n📊 Summary:")
        print(f"  Total time used: {time_budget - remaining_time}/{time_budget} minutes")
        print(f"  Total information gain: {total_gain:.2f}")
        print(f"  Time efficiency: {(time_budget - remaining_time)/time_budget*100:.1f}%")
        
        return selected
    
    def adaptive_parameter_optimization(self, experiment_type, initial_params):
        """
        Use AI to adaptively optimize experimental parameters.
        """
        print(f"\n🧠 Adaptive Parameter Optimization for {experiment_type}")
        print("=" * 50)
        
        # Simulate iterative optimization
        current_params = initial_params.copy()
        optimization_trace = []
        
        for iteration in range(5):  # 5 optimization rounds
            # Simulate running experiment with current parameters
            fidelity = self._simulate_experiment_fidelity(experiment_type, current_params)
            
            # Log iteration
            optimization_trace.append({
                'iteration': iteration + 1,
                'params': current_params.copy(),
                'fidelity': fidelity
            })
            
            print(f"Iteration {iteration + 1}: Fidelity = {fidelity:.4f}")
            
            # AI-suggested parameter updates
            if iteration < 4:  # Don't update on last iteration
                current_params = self._suggest_parameter_updates(current_params, fidelity)
                
        # Find best parameters
        best_iteration = max(optimization_trace, key=lambda x: x['fidelity'])
        
        print(f"\n✨ Optimization Complete!")
        print(f"Best fidelity: {best_iteration['fidelity']:.4f} (iteration {best_iteration['iteration']})")
        print(f"Improvement: {best_iteration['fidelity'] - optimization_trace[0]['fidelity']:+.4f}")
        
        return best_iteration['params'], optimization_trace
    
    def _simulate_experiment_fidelity(self, exp_type, params):
        """Simulate experiment fidelity for given parameters."""
        # Base fidelity with some noise
        base_fidelity = 0.95
        
        # Parameter-dependent improvements/degradations
        if exp_type == 'rabi':
            # Optimal amplitude around 0.5
            amplitude_penalty = abs(params.get('amplitude', 0.5) - 0.5) * 0.1
            fidelity = base_fidelity - amplitude_penalty
        elif exp_type == 'ramsey':
            # Optimal frequency close to target
            freq_penalty = abs(params.get('frequency', 5000) - 5000) / 1000 * 0.05
            fidelity = base_fidelity - freq_penalty
        else:
            fidelity = base_fidelity
            
        # Add noise
        fidelity += np.random.normal(0, 0.005)
        return max(0, min(1, fidelity))  # Clamp between 0 and 1
    
    def _suggest_parameter_updates(self, current_params, current_fidelity):
        """AI-suggested parameter updates based on current performance."""
        new_params = current_params.copy()
        
        # Simple gradient-based optimization simulation
        for param, value in current_params.items():
            if param == 'amplitude':
                # Move towards optimal amplitude (0.5)
                step_size = 0.02
                if value > 0.5:
                    new_params[param] = value - step_size
                else:
                    new_params[param] = value + step_size
            elif param == 'frequency':
                # Random walk with bias towards improvement
                step_size = np.random.choice([-2, -1, 1, 2])
                new_params[param] = value + step_size
                
        return new_params

# Create and demonstrate AI experiment generator
ai_generator = AIExperimentGenerator(ai_qubit)

# Generate optimal experiment sequence
optimal_sequence = ai_generator.generate_optimal_sequence(
    time_budget=25,  # 25 minutes available
    objectives=['calibration', 'characterization']
)

log_and_record("ai_experiment_sequence", {
    "sequence": optimal_sequence,
    "time_budget": 25,
    "objectives": ['calibration', 'characterization']
})

## Intelligent Calibration Workflows

AI-optimized calibration procedures use machine learning to:

1. **Predict Parameter Drift**: Anticipate when recalibration is needed
2. **Adaptive Scheduling**: Adjust calibration frequency based on system stability
3. **Parameter Correlation**: Learn relationships between different calibration parameters
4. **Failure Prediction**: Identify early warning signs of calibration failure
5. **Resource Allocation**: Optimize calibration time vs. accuracy trade-offs

### Intelligent Calibration Manager

The AI calibration system continuously learns from your quantum system's behavior to optimize calibration protocols.

In [ ]:
# Demonstrate adaptive parameter optimization with Rabi experiment
print("=" * 70)
print("AI ADAPTIVE PARAMETER OPTIMIZATION DEMO")
print("=" * 70)

# Example: Optimize Rabi experiment parameters
initial_rabi_params = {
    'amplitude': 0.3,  # Starting point away from optimal
    'frequency': 5000,
    'pulse_width': 0.05
}

print("Starting Rabi parameter optimization...")
optimal_rabi_params, rabi_trace = ai_generator.adaptive_parameter_optimization(
    'rabi', initial_rabi_params
)

print("\n" + "="*50)
print("AI INTELLIGENT CALIBRATION MANAGER")
print("="*50)

class IntelligentCalibrationManager:
    """
    AI-powered calibration optimization and scheduling.
    """
    
    def __init__(self, qubit_element):
        self.qubit = qubit_element
        self.calibration_history = []
        self.drift_model = None
        self.stability_score = 1.0
        
    def predict_parameter_drift(self, time_horizon_hours=8):
        """
        Predict parameter drift using AI model.
        """
        print(f"\n🔮 Parameter Drift Prediction (next {time_horizon_hours} hours)")
        print("=" * 55)
        
        # Simulate drift prediction model
        current_time = datetime.now()
        
        # Simulate different parameters with different drift patterns
        predictions = {
            'frequency': {
                'current': 5042.3,
                'predicted': 5042.3 + 0.5 * np.sin(time_horizon_hours/12 * np.pi),
                'confidence': 0.85,
                'drift_rate': 0.06  # MHz/hour
            },
            'amplitude': {
                'current': 0.5487,
                'predicted': 0.5487 + 0.002 * time_horizon_hours,
                'confidence': 0.92,
                'drift_rate': 0.0003  # per hour
            },
            't1': {
                'current': 85,
                'predicted': 85 - 0.5 * time_horizon_hours,
                'confidence': 0.73,
                'drift_rate': -0.5  # μs/hour
            }
        }
        
        for param, data in predictions.items():
            current = data['current']
            predicted = data['predicted']
            drift = predicted - current
            confidence = data['confidence']
            
            status = "🟢" if abs(drift) < 0.1 * current else "🟡" if abs(drift) < 0.2 * current else "🔴"
            
            print(f"  {param.upper()}:")
            print(f"    Current: {current:.4f}")
            print(f"    Predicted: {predicted:.4f} (Δ = {drift:+.4f})")
            print(f"    Confidence: {confidence:.0%} {status}")
            print()
            
        return predictions
    
    def adaptive_calibration_schedule(self):
        """
        Generate adaptive calibration schedule based on system stability.
        """
        print("\n📅 Adaptive Calibration Schedule")
        print("=" * 40)
        
        # Calculate system stability metrics
        stability_metrics = {
            'frequency_stability': 0.85,  # How stable is frequency
            'amplitude_stability': 0.92,  # How stable is amplitude
            'coherence_stability': 0.78,  # How stable are T1/T2
            'measurement_stability': 0.89  # How stable is readout
        }
        
        # Overall stability score
        overall_stability = np.mean(list(stability_metrics.values()))
        self.stability_score = overall_stability
        
        print(f"System Stability Score: {overall_stability:.2f}/1.00")
        print()
        
        # Generate adaptive schedule
        if overall_stability > 0.9:
            schedule_type = "Relaxed"
            calibration_interval = 8  # hours
            quick_check_interval = 2  # hours
        elif overall_stability > 0.8:
            schedule_type = "Standard"
            calibration_interval = 6
            quick_check_interval = 1.5
        else:
            schedule_type = "Intensive"
            calibration_interval = 4
            quick_check_interval = 1
            
        schedule = {
            'type': schedule_type,
            'full_calibration_interval': calibration_interval,
            'quick_check_interval': quick_check_interval,
            'stability_score': overall_stability
        }
        
        print(f"Recommended Schedule: {schedule_type}")
        print(f"  Full calibration every: {calibration_interval} hours")
        print(f"  Quick checks every: {quick_check_interval} hours")
        
        return schedule
    
    def visualize_parameter_correlation(self):
        """
        Visualize correlations between calibration parameters.
        """
        print("\n🔗 Parameter Correlation Analysis")
        print("=" * 40)
        
        # Simulate correlation matrix
        parameters = ['frequency', 'amplitude', 't1', 't2', 'readout_amp']
        n_params = len(parameters)
        
        # Generate realistic correlation matrix
        correlations = np.array([
            [1.00,  0.12, -0.05,  0.08,  0.03],  # frequency
            [0.12,  1.00,  0.07,  0.15, -0.08],  # amplitude
            [-0.05, 0.07,  1.00,  0.65, -0.12],  # t1
            [0.08,  0.15,  0.65,  1.00, -0.18],  # t2
            [0.03, -0.08, -0.12, -0.18,  1.00]   # readout_amp
        ])
        
        # Create correlation heatmap
        fig = go.Figure(data=go.Heatmap(
            z=correlations,
            x=parameters,
            y=parameters,
            colorscale='RdBu',
            zmid=0,
            colorbar=dict(title="Correlation")
        ))
        
        fig.update_layout(
            title='Parameter Correlation Matrix',
            width=600,
            height=500
        )
        
        fig.show()
        
        # Identify strong correlations
        strong_correlations = []
        for i in range(n_params):
            for j in range(i+1, n_params):
                corr_val = correlations[i, j]
                if abs(corr_val) > 0.3:
                    strength = "Strong" if abs(corr_val) > 0.6 else "Moderate"
                    direction = "positive" if corr_val > 0 else "negative"
                    strong_correlations.append({
                        'param1': parameters[i],
                        'param2': parameters[j],
                        'correlation': corr_val,
                        'strength': strength,
                        'direction': direction
                    })
        
        print("\nSignificant Correlations:")
        for corr in strong_correlations:
            print(f"  {corr['param1']} ↔ {corr['param2']}: {corr['correlation']:+.3f} ({corr['strength']} {corr['direction']})")
            
        return correlations

# Create and demonstrate intelligent calibration manager
print("Creating Intelligent Calibration Manager...")

# Create a temporary qubit element for the calibration manager
qubit_config = {
    'hrid': 'AI_QUBIT',
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 5042.3,
            'channel': 1,
            'shape': 'blackman_drag',
            'amp': 0.5487,
            'phase': 0.,
            'width': 0.05,
            'alpha': 500,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9647.8,
            'channel': 1,
            'shape': 'square',
            'amp': 0.15,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        }
    }
}

ai_qubit_element = TransmonElement(name='AI_QUBIT', parameters=qubit_config)
cal_manager = IntelligentCalibrationManager(ai_qubit_element)

# Run AI-powered calibration analysis
drift_predictions = cal_manager.predict_parameter_drift()
schedule = cal_manager.adaptive_calibration_schedule()
correlations = cal_manager.visualize_parameter_correlation()

# Log intelligent calibration results
log_and_record("intelligent_calibration_analysis", {
    "drift_predictions": drift_predictions,
    "adaptive_schedule": schedule,
    "stability_score": cal_manager.stability_score
})

## LLM Integration for Experiment Planning

Large Language Models (LLMs) can significantly enhance quantum experiment workflows by:

1. **Natural Language Experiment Design**: Describe experiments in plain English
2. **Code Generation**: Automatically generate LeeQ experiment code
3. **Data Analysis**: Interpret experimental results and suggest next steps
4. **Documentation**: Generate experiment reports and documentation
5. **Troubleshooting**: Diagnose issues and suggest solutions
6. **Literature Integration**: Connect experimental results with published research

### LLM-Assisted Experiment Planning

The LLM integration allows you to interact with LeeQ using natural language, making quantum experiments more accessible.

In [ ]:
class LLMExperimentAssistant:
    """
    LLM-powered assistant for quantum experiment planning and analysis.
    """
    
    def __init__(self):
        self.conversation_history = []
        self.experiment_knowledge_base = self._load_knowledge_base()
        
    def _load_knowledge_base(self):
        """Load quantum experiment knowledge base."""
        return {
            'experiment_types': {
                'rabi': {
                    'description': 'Measure qubit control by varying drive amplitude',
                    'outputs': 'pi_pulse_amplitude, rabi_frequency',
                    'time_estimate': '5-10 minutes',
                    'prerequisites': 'rough_frequency_calibration'
                },
                't1': {
                    'description': 'Measure energy relaxation time',
                    'outputs': 'relaxation_time_t1',
                    'time_estimate': '10-15 minutes',
                    'prerequisites': 'pi_pulse_calibration'
                },
                'ramsey': {
                    'description': 'Measure dephasing time and fine-tune frequency',
                    'outputs': 't2_star, frequency_offset',
                    'time_estimate': '8-12 minutes',
                    'prerequisites': 'pi_half_pulse_calibration'
                }
            },
            'troubleshooting': {
                'low_contrast': ['check_measurement_parameters', 'verify_pulse_calibration', 'check_mixer_calibration'],
                'high_leakage': ['adjust_drag_parameters', 'reduce_pulse_amplitude', 'check_anharmonicity'],
                'unstable_frequency': ['check_temperature_stability', 'verify_flux_noise', 'recalibrate_more_frequently']
            }
        }
    
    def natural_language_query(self, query):
        """
        Process natural language queries about quantum experiments.
        """
        print(f"\n🤖 LLM Assistant: Processing query...")
        print(f"Query: \"{query}\"")
        print("=" * 60)
        
        # Simple keyword-based response system (simulating LLM)
        query_lower = query.lower()
        
        if 'rabi' in query_lower and ('run' in query_lower or 'perform' in query_lower):
            response = self._generate_rabi_experiment_code()
        elif 't1' in query_lower and ('measure' in query_lower or 'run' in query_lower):
            response = self._generate_t1_experiment_code()
        elif 'calibrate' in query_lower or 'calibration' in query_lower:
            response = self._generate_calibration_workflow()
        elif 'troubleshoot' in query_lower or 'problem' in query_lower:
            response = self._generate_troubleshooting_guide()
        elif 'analyze' in query_lower or 'results' in query_lower:
            response = self._generate_analysis_suggestions()
        else:
            response = self._generate_general_guidance(query_lower)
            
        # Log conversation
        self.conversation_history.append({
            'query': query,
            'response': response,
            'timestamp': datetime.now().isoformat()
        })
        
        return response
    
    def _generate_rabi_experiment_code(self):
        """Generate code for Rabi experiment."""
        code_example = '''# AI-Generated Rabi Experiment Code
from leeq.experiments.builtin.basic.calibrations import RabiAmplitudeCalibration

# Create Rabi experiment with optimized parameters
rabi_experiment = RabiAmplitudeCalibration(
    name="AI_Rabi_Calibration",
    qubit=1,
    drive_frequency=ai_qubit.qubit_frequency,  # Use current best estimate
    amplitude_start=0.0,
    amplitude_stop=1.0,
    amplitude_points=31,  # Good resolution without excessive time
    pulse_width=0.05,  # Standard 50ns pulse
    repeated_measurement_count=1000  # High statistics for accurate calibration
)

# Run the experiment
results = rabi_experiment.run()

# Extract π-pulse amplitude
pi_amplitude = results['sweep_values'][np.argmax(results['measurement_probabilities'])]
print(f"Calibrated π-pulse amplitude: {pi_amplitude:.4f}")'''
        
        return {
            'response_type': 'code_generation',
            'explanation': "I've generated a Rabi experiment optimized for your system. This will sweep drive amplitude to find the π-pulse amplitude needed for reliable qubit control.",
            'code': code_example,
            'estimated_time': '5-8 minutes',
            'next_steps': ['Run the generated code', 'Analyze Rabi oscillations', 'Update pulse calibration']
        }
    
    def _generate_t1_experiment_code(self):
        """Generate code for T1 measurement."""
        code_example = '''# AI-Generated T1 Measurement Code
from leeq.experiments.builtin.basic.characterizations import T1Measurement
from scipy.optimize import curve_fit

# T1 experiment with AI-optimized parameters
t1_experiment = T1Measurement(
    name="AI_T1_Characterization",
    qubit=1,
    delay_start=0.0,
    delay_stop=200.0,  # Generous range for reliable fit
    delay_points=31,
    pi_pulse_amplitude=pi_amplitude,  # Use calibrated amplitude
    repeated_measurement_count=800
)

# Run measurement
results = t1_experiment.run()

# AI-assisted fitting
def exponential_decay(t, amplitude, t1, offset):
    return amplitude * np.exp(-t / t1) + offset

# Fit T1 decay
popt, _ = curve_fit(exponential_decay, results['sweep_values'], 
                   results['measurement_probabilities'])
                   
t1_measured = popt[1]
print(f"Measured T1: {t1_measured:.1f} μs")'''
        
        return {
            'response_type': 'code_generation',
            'explanation': "Here's a T1 measurement protocol optimized for your qubit parameters. The delay range is set based on your qubit's expected T1 time.",
            'code': code_example,
            'estimated_time': '10-12 minutes',
            'next_steps': ['Verify exponential decay fit', 'Compare with specification', 'Track T1 over time']
        }
    
    def _generate_calibration_workflow(self):
        """Generate comprehensive calibration workflow."""
        return {
            'response_type': 'workflow_guidance',
            'explanation': "Here's an AI-optimized calibration workflow for your quantum system:",
            'workflow_steps': [
                {'step': 1, 'task': 'Resonator Spectroscopy', 'time': '3 min', 'priority': 'High'},
                {'step': 2, 'task': 'Qubit Spectroscopy', 'time': '5 min', 'priority': 'High'},
                {'step': 3, 'task': 'Rabi Calibration', 'time': '6 min', 'priority': 'High'},
                {'step': 4, 'task': 'Ramsey Fine-tuning', 'time': '4 min', 'priority': 'Medium'},
                {'step': 5, 'task': 'DRAG Optimization', 'time': '12 min', 'priority': 'Medium'},
                {'step': 6, 'task': 'T1/T2 Characterization', 'time': '15 min', 'priority': 'Low'}
            ],
            'total_time': '45 minutes',
            'optimization_notes': [
                "Steps 1-3 are critical and should be done first",
                "DRAG optimization can be skipped if leakage is < 1%",
                "T1/T2 can be done less frequently (daily vs hourly)"
            ]
        }
    
    def _generate_troubleshooting_guide(self):
        """Generate troubleshooting recommendations."""
        return {
            'response_type': 'troubleshooting',
            'common_issues': [
                {
                    'issue': 'Low Rabi Contrast',
                    'symptoms': ['Excited state probability < 0.8', 'Noisy oscillations'],
                    'solutions': ['Check mixer calibration', 'Verify pulse amplitude', 'Check T1 time'],
                    'priority': 'High'
                },
                {
                    'issue': 'Frequency Instability',
                    'symptoms': ['Ramsey fringes drifting', 'Inconsistent spectroscopy'],
                    'solutions': ['Check temperature stability', 'Increase calibration frequency', 'Check flux noise'],
                    'priority': 'Medium'
                },
                {
                    'issue': 'High Measurement Error',
                    'symptoms': ['Poor state discrimination', 'Low readout SNR'],
                    'solutions': ['Optimize readout amplitude', 'Check resonator frequency', 'Verify integration time'],
                    'priority': 'High'
                }
            ],
            'diagnostic_steps': [
                "1. Run basic measurement statistics",
                "2. Check Rabi oscillations",
                "3. Verify resonator frequency",
                "4. Measure coherence times"
            ]
        }
    
    def _generate_analysis_suggestions(self):
        """Generate data analysis suggestions."""
        return {
            'response_type': 'analysis_guidance',
            'analysis_checklist': [
                "📊 Verify fit quality (R² > 0.95 for exponential decays)",
                "📈 Check for systematic trends in residuals",
                "🔍 Compare results with previous calibrations",
                "⚠️  Flag any parameters outside expected ranges",
                "📝 Document any anomalies for future reference"
            ],
            'visualization_tips': [
                "Use log-scale plots for exponential decays",
                "Plot residuals to check fit quality",
                "Show error bars for statistical uncertainty",
                "Include reference lines for specifications"
            ],
            'reporting_suggestions': [
                "Include experimental parameters and conditions",
                "Report uncertainties with appropriate precision",
                "Add trend analysis for time-series data",
                "Note any deviations from expected behavior"
            ]
        }
    
    def _generate_general_guidance(self, query):
        """Generate general guidance for unrecognized queries."""
        return {
            'response_type': 'general_guidance',
            'message': "I can help you with quantum experiments! Try asking about:",
            'capabilities': [
                "🔬 Running specific experiments (Rabi, T1, T2, etc.)",
                "🛠️  Calibration procedures and workflows",
                "🔧 Troubleshooting experimental issues",
                "📊 Data analysis and visualization",
                "📚 Best practices and optimization tips"
            ],
            'example_queries': [
                "\"Run a Rabi experiment on qubit 1\"",
                "\"How do I calibrate my system?\"",
                "\"My Rabi contrast is low, what should I check?\"",
                "\"Analyze my T1 measurement results\""
            ]
        }
    
    def display_response(self, response):
        """Display formatted response."""
        if response['response_type'] == 'code_generation':
            print(f"💡 {response['explanation']}")
            print(f"\n⏱️  Estimated time: {response['estimated_time']}")
            print(f"\n```python")
            print(response['code'])
            print(f"```")
            print(f"\n📋 Next steps:")
            for step in response['next_steps']:
                print(f"  • {step}")
                
        elif response['response_type'] == 'workflow_guidance':
            print(f"💡 {response['explanation']}")
            print(f"\n📋 Workflow Steps (Total: {response['total_time']}):")
            for step in response['workflow_steps']:
                priority_emoji = "🔴" if step['priority'] == 'High' else "🟡" if step['priority'] == 'Medium' else "🟢"
                print(f"  {step['step']}. {step['task']} ({step['time']}) {priority_emoji}")
            print(f"\n💡 Optimization Notes:")
            for note in response['optimization_notes']:
                print(f"  • {note}")
                
        elif response['response_type'] == 'troubleshooting':
            print(f"🔧 Common Issues and Solutions:")
            for issue in response['common_issues']:
                priority_emoji = "🔴" if issue['priority'] == 'High' else "🟡"
                print(f"\n{priority_emoji} {issue['issue']}:")
                print(f"  Symptoms: {', '.join(issue['symptoms'])}")
                print(f"  Solutions: {', '.join(issue['solutions'])}")
            print(f"\n🔍 Diagnostic Steps:")
            for step in response['diagnostic_steps']:
                print(f"  {step}")
                
        elif response['response_type'] == 'analysis_guidance':
            print(f"📊 Analysis Checklist:")
            for item in response['analysis_checklist']:
                print(f"  {item}")
            print(f"\n📈 Visualization Tips:")
            for tip in response['visualization_tips']:
                print(f"  • {tip}")
            print(f"\n📝 Reporting Suggestions:")
            for suggestion in response['reporting_suggestions']:
                print(f"  • {suggestion}")
                
        else:  # general_guidance
            print(f"💡 {response['message']}")
            print(f"\n🔧 My Capabilities:")
            for capability in response['capabilities']:
                print(f"  {capability}")
            print(f"\n💬 Example Queries:")
            for example in response['example_queries']:
                print(f"  {example}")

# Create and demonstrate LLM assistant
llm_assistant = LLMExperimentAssistant()

print("\n" + "#"*70)
print("#" + " "*23 + "LLM EXPERIMENT ASSISTANT" + " "*22 + "#")
print("#"*70)

# Example interactions
queries_and_responses = [
    "I need to run a Rabi experiment to calibrate my pi pulse amplitude",
    "What's the best calibration workflow for my qubit?",
    "My Rabi contrast is really low, help me troubleshoot this problem"
]

for i, query in enumerate(queries_and_responses, 1):
    print(f"\n{'='*70}")
    print(f"EXAMPLE {i}: Natural Language Query")
    print('='*70)
    
    response = llm_assistant.natural_language_query(query)
    llm_assistant.display_response(response)
    
# Log LLM interactions
log_and_record("llm_assistant_interactions", {
    "interactions": llm_assistant.conversation_history,
    "query_types": ["rabi_calibration", "workflow_guidance", "troubleshooting"]
})

print("\n" + "#"*70)
print("# AI INTEGRATION TUTORIAL COMPLETE - All systems demonstrated! #")
print("#"*70)

## Conclusion

Congratulations! You've completed the LeeQ tutorial series. You now have a solid foundation in:

- LeeQ core concepts and simulation
- Single and multi-qubit experiments
- Complete calibration workflows
- AI-assisted experiment generation

## Next Steps

Explore the following resources for deeper learning:

- **Examples**: Check out `/notebooks/examples/` for specific experiment implementations
- **Workflows**: See `/notebooks/workflows/` for complete experimental procedures
- **Documentation**: Visit the full documentation for detailed API references